In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [64]:
iris_df = pd.read_csv('../dataset/iris.data', names=['feature_1', 'feature_2', 'feature_3', 'feature_4', 'label'], index_col=False)

In [65]:
iris_df.head()

,feature_1,feature_2,feature_3,feature_4,label
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [66]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
iris_df['label'] = le.fit_transform(iris_df['label'])

# Label Encoding Guidelines
* setosa = 0
* versicolor = 1
* virginica = 2

In [67]:
from sklearn.cluster import AgglomerativeClustering

In [68]:
agg = AgglomerativeClustering(3)

In [69]:
features = iris_df.drop(['label'], axis=1)
agg.fit(features)

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='ward', memory=None, n_clusters=3,
                        pooling_func='deprecated')

In [70]:
agg.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2,
       2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0], dtype=int64)

In [71]:
def single_link(features, clusters, p_a):
    new_distance = []
    list_f = features.values.tolist()
    for cluster in clusters :
        init_a = list_f[cluster[0]]
        init_b = list_f[clusters[p_a][0]]
        min_distance = np.linalg.norm(np.array(init_a)-np.array(init_b))
        for a in cluster :
            for b in clusters[p_a] :
                new_dist = np.linalg.norm(np.array(list_f[a])-np.array(list_f[b]))
                min_distance = new_dist if new_dist < min_distance else min_distance 
        new_distance.append(min_distance)
    return new_distance

In [72]:
def complete_link(features, clusters, p_a):
    new_distance = []
    list_f = features.values.tolist()
    for cluster in clusters :
        init_a = list_f[cluster[0]]
        init_b = list_f[clusters[p_a][0]]
        max_distance = np.linalg.norm(np.array(init_a)-np.array(init_b))
        for a in cluster :
            for b in clusters[p_a] :
                new_dist = np.linalg.norm(np.array(list_f[a])-np.array(list_f[b]))
                max_distance = new_dist if new_dist > max_distance else max_distance 
        new_distance.append(max_distance)
    return new_distance

In [73]:
def avg_list(p):
    sum_feature = [0 for i in range(len(p[0]))]
    for i in range(len(p[0])):
        for j in range(len(p)):
            sum_feature[i] = sum_feature[i] + p[j][i]
    sum_feature[:] = [x / len(p) for x in sum_feature]
    
    return sum_feature
    
def create_feature_list(features, cluster):
    feature_list = []
    list_f = features.values.tolist()

    for p in cluster:
        feature_list.append(list_f[p])
    
    return feature_list
    
def average_link(features, clusters, p_a):
    new_distance = []
    feature_list_a = create_feature_list(features, clusters[p_a])

    for cluster in clusters:
        feature_list_b = create_feature_list(features, cluster)
        distance = np.linalg.norm( np.array(avg_list(feature_list_a)) - np.array(avg_list(feature_list_b)) )
        new_distance.append(distance)

    return new_distance


In [74]:
def average_complete_link() :
    new_distance = []
    feature_list_a = create_feature_list(features, clusters[p_a])

    for cluster in clusters:
        feature_list_b = create_feature_list(features, cluster)
        distance = 0
        for i in range(len(feature_list_a)):
            for j in range(len(feature_list_b)):
                distance += np.linalg.norm( np.array(feature_list_a[i]) - np.array(feature_list_b[j]) )
                
        new_distance.append(distance / (len(feature_list_a)*len(feature_list_b)))

    return new_distance

In [75]:
def find_adj_matrix(features) :
    
    adj_matrix = [[0 for x in range(features.shape[0])] for y in range(features.shape[0])]
    
    for a in range(features.shape[0]) :
        for b in range(features.shape[0]) :
            if (b <= a) :
                adj_matrix[a][b] = np.linalg.norm(features.iloc[a] - features.iloc[b])
                adj_matrix[b][a] = adj_matrix[a][b]
            else :
                break
                
    return adj_matrix


In [76]:
def find_cluster_pair(adj_matrix) :
    idx_min_a, idx_min_b = 1,0
    closest_distance = adj_matrix [1][0]
    for i in range(len(adj_matrix)) :
        for j in range(len(adj_matrix)) :
            if (not i == j) and (j <= i) :
                if (adj_matrix[i][j] < closest_distance) :
                    closest_distance = adj_matrix[i][j]
                    idx_min_a, idx_min_b = i,j
    
    return (idx_min_a,idx_min_b, closest_distance)
        

In [77]:
def update_cluster(p_1, p_2, cluster) :
    new_cluster = cluster[p_2][:]
    new_cluster.extend(cluster[p_1])
    
    cluster.pop(p_1)
    cluster.pop(p_2)
    cluster.insert(p_2, new_cluster)
    
    return cluster
    

In [78]:
def update_adj_matrix(features, p_1, p_2, cluster, cluster_matrix, linkage) :
    cluster_matrix.pop(p_1)
    cluster_matrix.pop(p_2)

    new_distance = []
    if (linkage == 1):
        new_distance = single_link(features, cluster, p_2)
    elif (linkage == 2):
        new_distance = complete_link(features, cluster, p_2)
    elif (linkage == 3):
        new_distance = average_link(features, cluster, p_2)
        
    cluster_matrix.insert(p_2,new_distance)
    for i in range(len(cluster_matrix)) :
        cluster_matrix[i][p_2] = new_distance[i]
    return cluster_matrix

In [79]:
'''
N_cluster = number of cluster

linkage (1) = Single
linkage (2) = Complete
linkage (3) = Average
linkage (4) = Average-Group

'''

def agglomerative(features, n_cluster, linkage) :

    cluster = [[i] for i in range(features.shape[0])]
    adj_matrix = find_adj_matrix(features)
    cluster_matrix = adj_matrix[:][:]
    number_cluster = len(cluster)
    while number_cluster > n_cluster :
        p_1, p_2, distance = find_cluster_pair(cluster_matrix)
        cluster = update_cluster(p_1, p_2, cluster)
        cluster_matrix = update_adj_matrix(features, p_1, p_2, cluster, cluster_matrix, linkage)
        number_cluster = len(cluster)
    return cluster

linkage = 2
n_cluster = 3
cluster = agglomerative(features, n_cluster, linkage)
print(cluster)


[[0, 17, 4, 7, 39, 27, 28, 26, 31, 23, 20, 21, 10, 48, 30, 11, 6, 19, 1, 45, 12, 9, 34, 37, 35, 2, 3, 47, 5, 42, 13, 29, 8, 38, 49, 44, 22, 14, 16, 36, 24, 33, 25, 46, 32, 43, 15, 18, 40, 41], [50, 52, 54, 51, 56, 58, 66, 78, 85, 62, 70, 134, 59, 69, 74, 75, 80, 119, 133, 90, 97, 83, 111, 113, 123, 53, 110, 55, 61, 67, 89, 88, 96, 95, 99, 71, 77, 82, 94, 91, 92, 68, 57, 79, 84, 103, 142, 64, 93, 129, 149, 60, 76, 86, 63, 65, 72, 73, 87, 138, 101, 121, 104, 112, 132, 141, 115, 140, 144, 116, 124, 137, 143, 148, 120, 126, 139, 108, 81, 98, 102, 127], [100, 105, 107, 122, 125, 130, 135, 106, 109, 114, 118, 117, 128, 136, 145, 146, 131, 147]]


In [80]:
def set_label(cluster, features) :
    predicted_label = [0 for i in range(features.shape[0])]

    for i in range(len(cluster)) :
        for x in cluster[i] :
            predicted_label[x] = i
    predicted_label = np.array(predicted_label)        
    return predicted_label

In [81]:
predicted_label = set_label(cluster, features)

In [82]:
from scipy.stats import mode
from sklearn.metrics import confusion_matrix, accuracy_score

y = iris_df['label'].values
def calculate_accuracy(y_truth, y_predicted):
    labels = np.zeros_like(y_predicted)
    for i in range(3):
        mask = (y_predicted == i)
        labels[mask] = mode(y_truth[mask])[0]
    return accuracy_score(y_truth, labels)

In [83]:
calculate_accuracy(y, agg.labels_)

0.8933333333333333

In [84]:
calculate_accuracy(y, predicted_label)

0.7866666666666666